# 코드 소개
● 자신의 업비트 API키와 바이낸스 API키를 보유하고있다면 실행가능한 코드

● 업비트의 공지를 설정한 time_sleep마다 불러와 '디지털 자산 추가' 라는 공지가 확인되면 상장된 코인을 바이낸스 선물에서 Long 포지션 및 Short 포지션으로 설정한 수량만큼 매수하는 코드

#주요 기능
● 업비트 API(https://api-manager.upbit.com/api/v1/notices)를 사용하여 최신 공지사항 데이터를 가져옴

● 공지사항 제목에서 괄호로 감싸진 디지털 자산 이름을 추출

● 추가된 자산이 관심 목록(my_coins 리스트)에 있는지 확인

● 바이낸스 API를 사용하여 잔고 조회 및 마켓 주문을 생성

● 0.5초 간격으로 반복 실행하여 업비트 공지사항을 실시간으로 모니터링

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import ccxt

#업비트 API키와 바이낸스 API키 설정
with open("api.txt") as f:
    lines = f.readlines()
    api_key = lines[0].strip()
    secret = lines[1].strip()

binance = ccxt.binance(config={
    'apiKey': api_key,
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future',
        'leverage' : 3
    }
})

sybmol = ""
my_balance = binance.fetch_balance(params = {"type" : "future"}) #현물이 아닌 선물로 설정
balance = my_balance['USDT']['total']

my_coins = ["YFIUSDT" , "MKRUSDT" , "DEFIUSDT" , "FOOTBALLUSDT" , "BNBUSDT" , "XMRUSDT" , "QNTUSDT" , "LTCUSDT" , "TRBUSDT" ,
"COMPUSDT" , "GMXUSDT" , "DASHUSDT" , "ZECUSDT" ,
"KSMUSDT" , "SSVUSDT" , "NMRUSDT" , "ZENUSDT" , "INJUSDT" , "UNFIUSDT" , "BLUEBIRDUSDT" , "LPTUSDT" , "FXSUSDT" , "XVSUSDT" , "CYBERUSDT", "ANTUSDT", "UNIUSDT",
"ARUSDT", "FILUSDT", "BALUSDT" , "ICPUSDT", "CVXUSDT" , "RUNEUSDT", "SNXUSDT" , "DYDXUSDT", "RNDRUSDT",
"LDOUSDT" , "WLDUSDT", "TOMOUSDT", "UMAUSDT", "OPUSDT", "RADUSDT", "HIGHUSDT","GALUSDT","LQTYUSDT","APEUSDT",
"BANDUSDT","API3USDT" , "RLCUSDT", "GTCUSDT" ,"HOOKUSDT", "LITUSDT", "ALICEUSDT",
"PENDLEUSDT" ,  "PHBUSDT" ,"BELUSDT", "COMBOUSDT", "SFPUSDT", "SUSHIUSDT",
"PERPUSDT", "MAGICUSDT", "CRVUSDT", "OMGUSDT", "STGUSDT", "EDUUSDT","CTKUSDT",
"CELOUSDT", "BNTUSDT", "ARKMUSDT",  "HFTUSDT", "FRONTUSDT", "OCEANUSDT",
"JOEUSDT" , "RDNTUSDT", "YGGUSDT" , "BICOUSDT", "ENJUSDT", "FETUSDT" , "BNXUSDT" ,  "FTMUSDT",
"IDUSDT" ,"WOOUSDT", "AGIXUSDT" , "LRCUSDT" ,
"C98USDT" , "AUDIOUSDT" , "BLZUSDT" , "CFXUSDT" , "BAKEUSDT", "CTSIUSDT", "OGNUSDT",  "KLAYUSDT" , "XLMUSDT", "DUSKUSDT",
"DODOXUSDT" , "CHRUSDT" , "DARUSDT" ,  "NKNUSDT" , "ATAUSDT" , "FLMUSDT" , "ALPHAUSDT", "OXTUSDT",
"IDEXUSDT" , "MDTUSDT" , "RENUSDT", "ARPAUSDT" ,
"TRUUSDT" , "COTIUSDT" , "XEMUSDT" , "SKLUSDT" , "1000FLOKIUSDT" , "IOTXUSDT" ,
"GALAUSDT" , "RVNUSDT" , "ACHUSDT" , "CELRUSDT", "LINAUSDT" , "AMBUSDT" , "TLMUSDT", "PEOPLEUSDT" , "ONEUSDT" ,
"DGBUSDT" , "KEYUSDT" , "XVGUSDT" , "JASMYUSDT" , "CKBUSDT" , "RSRUSDT" , "LEVERUSDT", "REEFUSDT" , "HOTUSDT" , "1000PEPEUSDT",
"DENTUSDT" , "SPELLUSDT"]

while True:
    url = "https://api-manager.upbit.com/api/v1/notices?page=1&per_page=20&thread_name=general"
    response = requests.get(url)
    data = response.json()

    latest_notice_title = data['data']['list'][0]['title'] #업비트의 최신 공지 불러오기

    #latest_notice_title ="[거래] KRW 마켓 디지털 자산 추가 (CKB)" #테스트할때 앞에 맨앞에(latest앞 #) #삭제하고 ()안에 코인 넣어서 테스트
    if "[거래] KRW 마켓 디지털 자산 추가" in latest_notice_title:
        match = re.search(r'\((.*?)\)', latest_notice_title)

        if match:  #해당 공지가 인식되었을 시
            added_digital_asset = match.group(1)
            print(f"추가된 디지털 자산: {added_digital_asset}")
            symbol = added_digital_asset + "USDT"
            print(symbol)

        if symbol in my_coins: #롱으로 매수할 시
            coin_price = binance.fetch_ticker(symbol)
            coin_last_price = coin_price['last']
            buy_amount = coin_last_price * (balance * 0.5) #코인 가격 x 내가가진 잔고 x 0.5
            binance.create_market_buy_order(symbol = symbol , amount = buy_amount)
            break

        #if symbol in my_coins: # 숏으로 매수 시 주석 삭제 후 사용 / 위에 if symbol in coins 부터 break까지 줄은 ctrl + / 으로 주석처리
            #coin_price = binance.fetch_ticker(symbol)
            #coin_last_price = coin_price['last']
            #buy_amount = coin_last_price * (balance * 0.5) #코인 가격 x 내가가진 잔고 x 0.5
            #binance.create_market_sell_order(symbol=symbol , amount = buy_amount)
            #break
        else:
            print("디지털 자산 없음.")
    else:
        print(latest_notice_title)

    print(balance)
    time.sleep(0.5)